In [0]:
%load_ext autoreload

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Bert

In [0]:
import keras
!pip install keras_bert
from keras_bert import load_vocabulary, load_trained_model_from_checkpoint, Tokenizer, get_checkpoint_paths
from keras_bert.layers import MaskedGlobalMaxPool1D
import numpy as np
import re

Using TensorFlow backend.


  Created wheel for keras-bert: filename=keras_bert-0.80.0-cp36-none-any.whl size=37923 sha256=9ab2dc70ad12cf3394116dc75aa19eb1c6b984b8a090f3aefb8665eba33b8d0b
  Stored in directory: /root/.cache/pip/wheels/63/dc/87/3260cb91f3aa32c0f85c5375429a30c8fd988bbb48f5ee21b0
  Created wheel for keras-transformer: filename=keras_transformer-0.31.0-cp36-none-any.whl size=13385 sha256=ccc8c1cfe1cb935e406f48a21489c9a246a7a1b7d53f2f1ada1d2d69a31a6629
  Stored in directory: /root/.cache/pip/wheels/a3/c5/9a/5a5130240be614a7a6fa786765d7692ae97f82601e2161bb56
  Created wheel for keras-pos-embd: filename=keras_pos_embd-0.11.0-cp36-none-any.whl size=7553 sha256=c35ca803d369a8f48c60ffd5235057a9a22512de02f8ea5762b85e4c32b24ca8
  Stored in directory: /root/.cache/pip/wheels/5b/a1/a0/ce6b1d49ba1a9a76f592e70cf297b05c96bc9f418146761032
  Created wheel for keras-multi-head: filename=keras_multi_head-0.22.0-cp36-none-any.whl size=15371 sha256=ed7ac3d68b280d04797a04d48094ae33a59ac3367f12f10c31cbb0cf4e32b273
  Stor

In [0]:
model_path = '/content/drive/My Drive/bert'
paths = get_checkpoint_paths(model_path)

In [0]:
seq_len = 50
bert_model = load_trained_model_from_checkpoint(config_file=paths.config, checkpoint_file=paths.checkpoint, seq_len=seq_len)





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.









In [0]:
pool_layer = MaskedGlobalMaxPool1D(name='Pooling')(bert_model.output)

In [0]:
bert_model = keras.models.Model(inputs=bert_model.inputs, outputs=pool_layer)

In [0]:
token_dict = load_vocabulary(paths.vocab)
tokenizer = Tokenizer(token_dict)

In [0]:
def get_bert_vector(doc, bert_model, token_dict, seq_len):
    tokenized_doc = tokenizer.tokenize(doc)[:seq_len]
    segments = [0] * seq_len
    indices = [token_dict[elem] for elem in tokenized_doc]
    indices += [0] * (seq_len - len(indices))
    return bert_model.predict([np.array([indices]), np.array([segments])])[0]

In [0]:
from sklearn.metrics.pairwise import cosine_similarity

In [0]:
with open('/content/drive/My Drive/models/all_pairs.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [0]:
sents = [re.sub('\s+', ' ', re.sub('(?:[^ ])\d+', ' ', sent)).strip() for sent in text.split('\n') if sent.strip()]

In [0]:
len(sents)

363516

In [0]:
with open('all_sents.txt', 'w', encoding='utf-8') as f:
    f.write('\n'.join(sents))

In [0]:
bert_vector_size = 768
bert_vectors = np.zeros((len(sents), bert_vector_size))

for i, sent in enumerate(sents):
    if i % 1000 == 0:
        print(i)
    bert_vectors[i] = get_bert_vector(sent, bert_model, token_dict, seq_len)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000


In [0]:
def get_most_probable_docs(doc_vector, docs_matrix):
    cosine_values = cosine_similarity(docs_matrix, doc_vector.reshape(1, -1)).reshape(docs_matrix.shape[0])
    return [sents[sent_id] for sent_id, _ in sorted(list(
        enumerate(cosine_values)), key=lambda elem: elem[1], reverse=True)][0]

In [0]:
def search_bert(query, bert_vectors, bert_model, token_dict, seq_len):
    bert_vector = get_bert_vector(query, bert_model, token_dict, seq_len)
    return get_most_probable_docs(bert_vector, bert_vectors)

In [0]:
import h5py

In [0]:
print(bert_vectors.shape, type(bert_vectors))
h5f = h5py.File('/content/drive/My Drive/models/bert.h5', 'w')
h5f.create_dataset('dataset_1', data=bert_vectors)
h5f.close()

(363516, 768) <class 'numpy.ndarray'>


In [0]:
h5f = h5py.File('/content/drive/My Drive/models/bert.h5','r')
b = h5f['dataset_1'][:]
h5f.close()

True

In [0]:
!ls /content/drive/My\ Drive/models

all_pairs.gdoc	all_revised_toms.txt  bert.h5
all_pairs.txt	all_sents.txt	      model.hdf5


In [0]:
search_bert('Мне страшно жить здесь', bert_vectors, bert_model, token_dict, seq_len)

['Мне ужасно хотелось спать.',
 'И мне страшно.',
 'Мне жить хорошо.',
 'Мне тяжело.',
 'Мне так страшно!',
 'Мне жаль.',
 'Мне всё хочется плакать.',
 'Всегда страшно отпускать.',
 'А мне страшно.',
 'Тогда легко жить.',
 'Только мне ужасно хочется.',
 'Мне хочется взглянуть.',
 'Мне страшно за вас.',
 'А жить мне хорошо.',
 'Ему хочется написать.',
 'Так мне хорошо одному дома.',
 'И мне больно писать.',
 'Мне это ужасно нужно.',
 'Мне слишком тяжело.',
 'Мне надо показаться там.']

In [0]:
bert_vector_size = 768
bert_vectors = np.zeros((3, bert_vector_size))

# for i, tokenized_doc in enumerate(tokenized_docs):

bert_vectors[0] = get_bert_vector('Вчера поправлял коректуру об искусстве3 и всю перемарал (Урусов достал сына дьякона, кот орый сейчас переписывает), и в лесу рубил и пилил с мужиками, работающими там.', bert_model, token_dict, seq_len)
bert_vectors[1] = get_bert_vector('Дневники же, если я не успею более точно и ясно выразить то, что я записываю в них, могут иметь некоторое значение, хотя бы в тех отрывочных мыслях, которые изложены там.', bert_model, token_dict, seq_len)
bert_vectors[2] = get_bert_vector('А каждый день свое приносит омрачение.', bert_model, token_dict, seq_len)

In [0]:
print(cosine_similarity(bert_vectors[0].reshape(1, -1), bert_vectors[1].reshape(1, -1)))
print(cosine_similarity(bert_vectors[0].reshape(1, -1), bert_vectors[2].reshape(1, -1)))

[[0.9324744]]
[[0.87604891]]


Word2Vec

In [0]:
import gensim
!pip install pymystem3==0.1.10
from pymystem3 import Mystem
import zipfile

In [0]:
m = Mystem()

In [0]:
text = 'Мама мыла раму'

In [0]:
model_url = 'http://vectors.nlpl.eu/repository/11/180.zip'
m = wget.download(model_url)
model_file = model_url.split('/')[-1]
with zipfile.ZipFile(model_file, 'r') as archive:
    stream = archive.open('model.bin')
    model = gensim.models.KeyedVectors.load_word2vec_format(stream, binary=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
model.most_similar(positive=['хуйня_NOUN'], topn=20)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('фигня_NOUN', 0.5566369891166687),
 ('пиздец_NOUN', 0.5434011816978455),
 ('сраный_ADJ', 0.5090421438217163),
 ('херня_NOUN', 0.5032066106796265),
 ('черт-те_VERB', 0.4990184009075165),
 ('ерунда_NOUN', 0.49403128027915955),
 ('чушь_NOUN', 0.49284014105796814),
 ('пиздить_VERB', 0.49275171756744385),
 ('хрень_NOUN', 0.4916004240512848),
 ('чушь_VERB', 0.4886845350265503),
 ('блин_PROPN', 0.4870203137397766),
 ('хули_VERB', 0.4810410737991333),
 ('мудак_ADV', 0.4789344370365143),
 ('хер_NOUN', 0.4771116375923157),
 ('хуе_ADJ', 0.4762328267097473),
 ('дебил_NOUN', 0.47076573967933655),
 ('хреновина_NOUN', 0.4682707190513611),
 ('фуфло_NOUN', 0.4656623601913452),
 ('балда_NOUN', 0.46197065711021423),
 ('гроттерша_PROPN', 0.4600300192832947)]